# Step20 연산자 오버로드(1)
variable 인스턴스를 ndarray 인스턴스처럼 '보이게'하자!

##20.1 Mul 클래스 구현

In [2]:
import numpy as np
import weakref

class Variable:
  '''
  중간 변수에 대해서는 미분값을 제거하는 모드를 추가
  '''
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = name # name 추가
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output().grad for output in f.outputs] # output()(약한참조)의 grad 가져옴
      gxs = f.backward(*gys)                        # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가

      if not retain_grad: # retain_grad가 false 이면 중간 변수의 미분값을 모두 None으로 설정
        for y in f.outputs:
          y().grad = None # y는 약한 참조(weakref)

  # 다른 인스턴스 변수를 추가해 보자.
  # property 덕분에 메서드를 인스턴스 변수처럼 사용할 수 있음
  @property
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  # __len__ 구현
  def __len__(self):
    return len(self.data)

  # print 함수가 출력해주는 내용을 정의 -> __repr__메서드를 재정의하면됨
  def __repr__(self):
    if self.data is None:
      return 'variable(None)'

    p = str(self.data).replace('\n', '\n' + ' ' * 9)
    return 'variable(' + p + ')'

class Config:
  '''
    enable_backprop : 역전파가 가능한지 여부를 물음
    config 데이터는 한 곳에만 존재하는 것이 좋음.
    Config 클래스는 인스턴스화 하지 않고 '클래스' 상태로 이용!
    왜냐? 인스턴스는 여러 개 생성할 수 있지만, 클래스는 항상 하나만 존재하기 때문!
  '''
  enable_backprop = True

import contextlib

@contextlib.contextmanager
def using_config(name, value):
  old_value = getattr(Config, name) # Config의 'name'이름을 가진 속성을 가르킴
  setattr(Config, name, value) # 내가 원하는 value값으로 Config안의 name의 속성을 설정
  try:
    yield
  finally:
    setattr(Config, name, old_value) # 원래의 old value로 설정

def no_grad():
  '''
    단순히 using_config('enable_backprop', False) 코드를 호출해줌
  '''
  return using_config('enable_backprop', False)

class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs) # 언팩
    if not isinstance(ys, tuple): # tuple 형태가 아닐 경우 추가 지원
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop: # 클래스 상태의 enable_backprop = True 일때만 역전파 코드 실행
      self.generation = max([x.generation for x in inputs]) # 세대 설정
      for output in outputs:
        output.set_creator(self)   # 연결 설정
      self.inputs = inputs
      self.outputs= [weakref.ref(output) for output in outputs] 
    
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()


def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    '''
      backward 부분 추가
    '''
    return gy, gy

def add(x0, x1):
  return Add()(x0, x1)

class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.inputs[0].data # Function 함수에서 inputs (튜플)로 받기 때문에 input-> inputs[0]으로 바꿔줌
    gx = 2 * x * gy
    return gx

def square(x):
  return Square()(x)

In [3]:
class Mul(Function):
  def forward(self, x0, x1):
    y = x0 * x1
    return y

  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    return gy*x1, gy*x0

def mul(x0, x1):
  return Mul()(x0, x1)

In [4]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a,b), c)

y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


##20.2 연산자 오버로드

In [7]:
class Variable:
  '''
  중간 변수에 대해서는 미분값을 제거하는 모드를 추가
  '''
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = name # name 추가
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output().grad for output in f.outputs] # output()(약한참조)의 grad 가져옴
      gxs = f.backward(*gys)                        # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가

      if not retain_grad: # retain_grad가 false 이면 중간 변수의 미분값을 모두 None으로 설정
        for y in f.outputs:
          y().grad = None # y는 약한 참조(weakref)

  # 다른 인스턴스 변수를 추가해 보자.
  # property 덕분에 메서드를 인스턴스 변수처럼 사용할 수 있음
  @property
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  # __len__ 구현
  def __len__(self):
    return len(self.data)

  # print 함수가 출력해주는 내용을 정의 -> __repr__메서드를 재정의하면됨
  def __repr__(self):
    if self.data is None:
      return 'variable(None)'

    p = str(self.data).replace('\n', '\n' + ' ' * 9)
    return 'variable(' + p + ')'

  def __mul__(self, other):
    '''
    특수 메서드(__mum__ -> *로 호출 가능)를 정의 -> 사용자 지정함수(def mul)가 호출
    '''
    return mul(self, other)

In [8]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
y = a * b
print(y)

variable(6.0)


In [13]:
class Variable:
  '''
  중간 변수에 대해서는 미분값을 제거하는 모드를 추가
  '''
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = name # name 추가
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output().grad for output in f.outputs] # output()(약한참조)의 grad 가져옴
      gxs = f.backward(*gys)                        # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가

      if not retain_grad: # retain_grad가 false 이면 중간 변수의 미분값을 모두 None으로 설정
        for y in f.outputs:
          y().grad = None # y는 약한 참조(weakref)

  # 다른 인스턴스 변수를 추가해 보자.
  # property 덕분에 메서드를 인스턴스 변수처럼 사용할 수 있음
  @property
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  # __len__ 구현
  def __len__(self):
    return len(self.data)

  # print 함수가 출력해주는 내용을 정의 -> __repr__메서드를 재정의하면됨
  def __repr__(self):
    if self.data is None:
      return 'variable(None)'

    p = str(self.data).replace('\n', '\n' + ' ' * 9)
    return 'variable(' + p + ')'

# 앞의 def __mul__ 함수를 아래와 같이 Variable 밖에서 간단하게 설정 가능
Variable.__mul__ = mul
Variable.__add__ = add

In [14]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

# y = add(mul(a,b), c)
y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0
